# **Elaboration d'un modèle personnel**

## **Import des packages**

In [1]:
!pip install scikeras
!pip install scikit-learn
!pip install tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.4/13.4 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 347.7/347.7 kB 8.5 MB/s eta 0:00:00
  Attempting uninstall: scikit-learn
    Found existing installation: scikit-learn 1.2.2
    Uninstalling scikit-learn-1.2.2:
      Successfully uninstalled scikit-learn-1.2.2
  Attempting uninstall: keras
    Found existing installation: keras 2.15.0
    Uninstalling keras-2.15.0:
      Successfully uninstalled keras-2.15.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.15.0 requires keras<2.16,>=2.15.0, but you have keras 3.4.1 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 16.9 MB/s eta 0:00:00
  Attempting uninstall: keras
    Found existing installation: keras 3.4.1
    Un

In [13]:
import os
import shutil
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from scikeras.wrappers import KerasClassifier
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.model_selection import train_test_split, GridSearchCV, ParameterGrid
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

## **Téléchargement des données**

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
# Chemin vers le dossier contenant toutes les images
images_folder = '/content/drive/MyDrive/transformed_images'

# Chemins vers les dossiers de sortie sans augmentation
train_dir_no_aug = '/content/drive/MyDrive/train_no_aug'
val_dir_no_aug = '/content/drive/MyDrive/validation_no_aug'
test_dir_no_aug = '/content/drive/MyDrive/test_no_aug'

# Chemins vers les dossiers de sortie avec augmentation
train_dir_aug = '/content/drive/MyDrive/train_aug'
val_dir_aug = '/content/drive/MyDrive/validation_aug'
test_dir_aug = '/content/drive/MyDrive/test_aug'

# Créer les dossiers de sortie s'ils n'existent pas
for directory in [train_dir_no_aug, val_dir_no_aug, test_dir_no_aug, train_dir_aug, val_dir_aug, test_dir_aug]:
    os.makedirs(directory, exist_ok=True)

## **Selection des races pour l'entrainement**

In [5]:
selected_classes = ['n02106662-German_shepherd', 'n02093428-American_Staffordshire_terrier', 'n02116738-African_hunting_dog']

## **Séparation des données pour l'entrainement, la validation et le test**

In [6]:
def split_data(class_name, files, train_dir, val_dir, test_dir, train_ratio=0.7, val_ratio=0.2, test_ratio=0.1):
    np.random.shuffle(files)
    train_split = int(train_ratio * len(files))
    val_split = int((train_ratio + val_ratio) * len(files))

    train_files = files[:train_split]
    val_files = files[train_split:val_split]
    test_files = files[val_split:]

    for f in train_files:
        shutil.copy(f, os.path.join(train_dir, class_name, os.path.basename(f)))
    for f in val_files:
        shutil.copy(f, os.path.join(val_dir, class_name, os.path.basename(f)))
    for f in test_files:
        shutil.copy(f, os.path.join(test_dir, class_name, os.path.basename(f)))

for class_name in selected_classes:
    class_folder = os.path.join(images_folder, class_name)
    if os.path.isdir(class_folder):
        for directory in [train_dir_no_aug, val_dir_no_aug, test_dir_no_aug, train_dir_aug, val_dir_aug, test_dir_aug]:
            os.makedirs(os.path.join(directory, class_name), exist_ok=True)

        files = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
        split_data(class_name, files, train_dir_no_aug, val_dir_no_aug, test_dir_no_aug)
        split_data(class_name, files, train_dir_aug, val_dir_aug, test_dir_aug)

In [ ]:
for class_name in selected_classes:
    class_folder = os.path.join(images_folder, class_name)
    if os.path.isdir(class_folder):
        os.makedirs(os.path.join(train_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(val_dir, class_name), exist_ok=True)
        os.makedirs(os.path.join(test_dir, class_name), exist_ok=True)

        files = [os.path.join(class_folder, f) for f in os.listdir(class_folder) if os.path.isfile(os.path.join(class_folder, f))]
        split_data(class_name, files, train_dir, val_dir, test_dir)

## **Normalisation et Data augmentation**

In [7]:
# Data Augmentation pour l'entraînement
train_datagen_aug = ImageDataGenerator(
    rescale=1./255,
    rotation_range=40,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Normalisation pour validation et test
val_test_datagen = ImageDataGenerator(rescale=1./255)

# Générateurs de données pour les données avec augmentation
train_generator_aug = train_datagen_aug.flow_from_directory(
    train_dir_aug,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_generator_aug = val_test_datagen.flow_from_directory(
    val_dir_aug,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_generator_aug = val_test_datagen.flow_from_directory(
    test_dir_aug,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

# Générateurs de données pour les données sans augmentation
train_datagen_no_aug = ImageDataGenerator(
    rescale=1./255
)

train_generator_no_aug = train_datagen_no_aug.flow_from_directory(
    train_dir_no_aug,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

val_generator_no_aug = val_test_datagen.flow_from_directory(
    val_dir_no_aug,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical'
)

test_generator_no_aug = val_test_datagen.flow_from_directory(
    test_dir_no_aug,
    target_size=(128, 128),
    batch_size=32,
    class_mode='categorical',
    shuffle=False
)

Found 1467 images belonging to 3 classes.
Found 420 images belonging to 3 classes.
Found 213 images belonging to 3 classes.
Found 1467 images belonging to 3 classes.
Found 420 images belonging to 3 classes.
Found 213 images belonging to 3 classes.


## **Modélisation sans Data augmentation**

### **Recherche des meilleurs hyperparamètres**

In [8]:
def create_model(optimizer='adam', init_mode='uniform', dropout_rate=0.0):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu', kernel_initializer=init_mode),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Définir les hyperparamètres à tester
param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [10, 20],
    'optimizer': ['adam', 'sgd'],
    'init_mode': ['uniform', 'he_normal'],
    'dropout_rate': [0.0, 0.5]
}

# Convertir le param_grid en liste de dictionnaires
param_list = list(ParameterGrid(param_grid))

# Initialiser les meilleures performances et paramètres
best_accuracy = 0
best_params = None

# Boucle pour rechercher les meilleurs hyperparamètres
for params in param_list:
    print(f"Entraînement avec les paramètres : {params}")

    # Créer le modèle avec les hyperparamètres actuels
    model = create_model(optimizer=params['optimizer'], init_mode=params['init_mode'], dropout_rate=params['dropout_rate'])

    # Entraîner le modèle
    history = model.fit(
        train_generator_no_aug,
        steps_per_epoch=train_generator_no_aug.samples // params['batch_size'],
        epochs=params['epochs'],
        validation_data=val_generator_no_aug,
        validation_steps=val_generator_no_aug.samples // params['batch_size'],
        verbose=1
    )

    # Obtenir les meilleures performances de validation
    val_accuracy = max(history.history['val_accuracy'])

    # Mettre à jour les meilleures performances et paramètres si nécessaire
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_params = params

print(f"Meilleurs Hyperparamètres : {best_params}")
print(f"Meilleure Précision de Validation : {best_accuracy}")

Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'uniform', 'optimizer': 'adam'}
Epoch 1/10
 46/146 [========>.....................] - ETA: 1:46 - loss: 1.1096 - accuracy: 0.4213

146/146 [==============================] - 57s 374ms/step - loss: 1.1096 - accuracy: 0.4213 - val_loss: 1.0322 - val_accuracy: 0.5024
Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'uniform', 'optimizer': 'sgd'}
Epoch 1/10
 46/146 [========>.....................] - ETA: 1:48 - loss: 1.0752 - accuracy: 0.4110

146/146 [==============================] - 56s 371ms/step - loss: 1.0752 - accuracy: 0.4110 - val_loss: 1.0361 - val_accuracy: 0.5286
Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'he_normal', 'optimizer': 'adam'}
Epoch 1/10
 46/146 [========>.....................] - ETA: 1:57 - loss: 1.0717 - accuracy: 0.4492

146/146 [==============================] - 62s 403ms/step - loss: 1.0717 - accuracy: 0.4492 - val_loss: 0.8910 - val_accuracy: 0.6214
Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'he_normal', 'optimizer': 'sgd'}
Epoch 1/10
 46/146 [========>.....................] - ETA: 1:45 - loss: 1.0879 - accuracy: 0.4008

146/146 [==============================] - 61s 402ms/step - loss: 1.0879 - accuracy: 0.4008 - val_loss: 1.1066 - val_accuracy: 0.3500
Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 20, 'init_mode': 'uniform', 'optimizer': 'adam'}
Epoch 1/20
 46/146 [========>.....................] - ETA: 2:24 - loss: 1.0884 - accuracy: 0.4554

...

73/73 [==============================] - 56s 750ms/step - loss: 1.0961 - accuracy: 0.3661 - val_loss: 1.0892 - val_accuracy: 0.3548
Entraînement avec les paramètres : {'batch_size': 20, 'dropout_rate': 0.5, 'epochs': 20, 'init_mode': 'uniform', 'optimizer': 'adam'}
Epoch 1/20
46/73 [=================>............] - ETA: 27s - loss: 1.1242 - accuracy: 0.3756

73/73 [==============================] - 55s 734ms/step - loss: 1.1242 - accuracy: 0.3756 - val_loss: 1.0240 - val_accuracy: 0.6238
Entraînement avec les paramètres : {'batch_size': 20, 'dropout_rate': 0.5, 'epochs': 20, 'init_mode': 'uniform', 'optimizer': 'sgd'}
Epoch 1/20
46/73 [=================>............] - ETA: 27s - loss: 1.1038 - accuracy: 0.3579

73/73 [==============================] - 54s 716ms/step - loss: 1.1038 - accuracy: 0.3579 - val_loss: 1.0754 - val_accuracy: 0.4500
Entraînement avec les paramètres : {'batch_size': 20, 'dropout_rate': 0.5, 'epochs': 20, 'init_mode': 'he_normal', 'optimizer': 'adam'}
Epoch 1/20
46/73 [=================>............] - ETA: 29s - loss: 1.0723 - accuracy: 0.4260

73/73 [==============================] - 57s 755ms/step - loss: 1.0723 - accuracy: 0.4260 - val_loss: 1.0561 - val_accuracy: 0.5095
Entraînement avec les paramètres : {'batch_size': 20, 'dropout_rate': 0.5, 'epochs': 20, 'init_mode': 'he_normal', 'optimizer': 'sgd'}
Epoch 1/20
46/73 [=================>............] - ETA: 27s - loss: 1.0887 - accuracy: 0.3872

73/73 [==============================] - 59s 791ms/step - loss: 1.0887 - accuracy: 0.3872 - val_loss: 1.0794 - val_accuracy: 0.3929
Entraînement avec les paramètres : {'batch_size': 40, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'uniform', 'optimizer': 'adam'}
Epoch 1/10
36/36 [==============================] - 45s 1s/step - loss: 1.1697 - accuracy: 0.3993 - val_loss: 0.9409 - val_accuracy: 0.6250
Epoch 2/10
36/36 [==============================] - 43s 1s/step - loss: 0.8249 - accuracy: 0.6408 - val_loss: 0.7837 - val_accuracy: 0.7188
Epoch 3/10
36/36 [==============================] - 42s 1s/step - loss: 0.5694 - accuracy: 0.7629 - val_loss: 0.5748 - val_accuracy: 0.7875
Epoch 4/10
36/36 [==============================] - 42s 1s/step - loss: 0.3517 - accuracy: 0.8771 - val_loss: 0.4888 - val_accuracy: 0.7969
Epoch 5/10
36/36 [==============================] - 44s 1s/step - loss: 0.2295 - accuracy: 0.9276 - val_loss: 0.4079 - val_accuracy: 0.8719
Epoch 6/10
36/36 [================

### **Entraînement du modèle avec les meilleurs hyperparamètres**

In [9]:
model = create_model(optimizer=best_params['optimizer'], init_mode=best_params['init_mode'], dropout_rate=best_params['dropout_rate'])
history = model.fit(
    train_generator_no_aug,
    steps_per_epoch=train_generator_no_aug.samples // best_params['batch_size'],
    epochs=best_params['epochs'],
    validation_data=val_generator_no_aug,
    validation_steps=val_generator_no_aug.samples // best_params['batch_size'],
    verbose=1
)

Epoch 1/20
36/36 [==============================] - 44s 1s/step - loss: 1.1134 - accuracy: 0.4446 - val_loss: 0.9556 - val_accuracy: 0.5156
Epoch 2/20
36/36 [==============================] - 43s 1s/step - loss: 0.8158 - accuracy: 0.6530 - val_loss: 0.7399 - val_accuracy: 0.6906
Epoch 3/20
36/36 [==============================] - 41s 1s/step - loss: 0.5001 - accuracy: 0.8152 - val_loss: 0.5994 - val_accuracy: 0.7656
Epoch 4/20
36/36 [==============================] - 44s 1s/step - loss: 0.3016 - accuracy: 0.8884 - val_loss: 0.3897 - val_accuracy: 0.8875
Epoch 5/20
36/36 [==============================] - 44s 1s/step - loss: 0.1553 - accuracy: 0.9512 - val_loss: 0.3577 - val_accuracy: 0.8969
Epoch 6/20
36/36 [==============================] - 45s 1s/step - loss: 0.1075 - accuracy: 0.9695 - val_loss: 0.4333 - val_accuracy: 0.8844
Epoch 7/20
36/36 [==============================] - 44s 1s/step - loss: 0.0684 - accuracy: 0.9843 - val_loss: 0.3806 - val_accuracy: 0.9094
Epoch 8/20
36/36 [==

### **Evaluation du modèle**

In [10]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(test_generator_no_aug)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator_no_aug.classes

# Évaluation du modèle sur les données de test
test_loss, test_accuracy = model.evaluate(test_generator_no_aug, verbose=2)

# Afficher la précision du modèle sur les données de test
print(f'Précision sur les données de test : {test_accuracy*100:.2f}%')

7/7 [==============================] - 2s 294ms/step
7/7 - 2s - loss: 0.1998 - accuracy: 0.9531 - 2s/epoch - 312ms/step
Précision sur les données de test : 95.31%




## **Modélisation avec Data augmentation**



### **Recherche des meilleurs hyperparamètres**

In [12]:
def create_model(optimizer='adam', init_mode='uniform', dropout_rate=0.0):
    model = Sequential([
        Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(128, (3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu', kernel_initializer=init_mode),
        Dropout(dropout_rate),
        Dense(3, activation='softmax')
    ])
    model.compile(optimizer=optimizer,
                  loss='categorical_crossentropy',
                  metrics=['accuracy'])
    return model

# Définir les hyperparamètres à tester
param_grid = {
    'batch_size': [10, 20, 40],
    'epochs': [10, 20],
    'optimizer': ['adam', 'sgd'],
    'init_mode': ['uniform', 'he_normal'],
    'dropout_rate': [0.0, 0.5]
}

# Convertir le param_grid en liste de dictionnaires
param_list = list(ParameterGrid(param_grid))

# Initialiser les meilleures performances et paramètres
best_accuracy = 0
best_params = None

# Boucle pour rechercher les meilleurs hyperparamètres
for params in param_list:
    print(f"Entraînement avec les paramètres : {params}")

    # Créer le modèle avec les hyperparamètres actuels
    model = create_model(optimizer=params['optimizer'], init_mode=params['init_mode'], dropout_rate=params['dropout_rate'])

    # Entraîner le modèle
    history = model.fit(
        train_generator_aug,
        steps_per_epoch=train_generator_aug.samples // params['batch_size'],
        epochs=params['epochs'],
        validation_data=val_generator_aug,
        validation_steps=val_generator_aug.samples // params['batch_size'],
        verbose=1
    )

    # Obtenir les meilleures performances de validation
    val_accuracy = max(history.history['val_accuracy'])

    # Mettre à jour les meilleures performances et paramètres si nécessaire
    if val_accuracy > best_accuracy:
        best_accuracy = val_accuracy
        best_params = params

print(f"Meilleurs Hyperparamètres : {best_params}")
print(f"Meilleure Précision de Validation : {best_accuracy}")

Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'uniform', 'optimizer': 'adam'}
Epoch 1/10
 46/146 [========>.....................] - ETA: 2:00 - loss: 1.2083 - accuracy: 0.3817

146/146 [==============================] - 63s 414ms/step - loss: 1.2083 - accuracy: 0.3817 - val_loss: 1.0510 - val_accuracy: 0.4667
Entraînement avec les paramètres : {'batch_size': 10, 'dropout_rate': 0.0, 'epochs': 10, 'init_mode': 'uniform', 'optimizer': 'sgd'}
Epoch 1/10
 30/146 [=====>........................] - ETA: 2:13 - loss: 1.1049 - accuracy: 0.3602

KeyboardInterrupt: 

### **Entraînement du modèle avec les meilleurs hyperparamètres**

In [ ]:
model_aug = create_model(optimizer=best_params['optimizer'], init_mode=best_params['init_mode'], dropout_rate=best_params['dropout_rate'])
history = model.fit(
    train_generator_aug,
    steps_per_epoch=train_generator_aug.samples // best_params['batch_size'],
    epochs=best_params['epochs'],
    validation_data=val_generator_aug,
    validation_steps=val_generator_aug.samples // best_params['batch_size'],
    verbose=1
)

## **Evaluation du modèle avec Data augmentation**

In [ ]:
# Prédictions sur l'ensemble de test
y_pred = model.predict(test_generator_aug)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = test_generator_aug.classes

# Évaluation du modèle sur les données de test
test_loss, test_accuracy = model.evaluate(test_generator_aug, verbose=2)

# Afficher la précision du modèle sur les données de test
print(f'Précision sur les données de test : {test_accuracy*100:.2f}%')


## **Affichage de la matrice de confusion du meilleur modèle**

In [ ]:
# Matrice de confusion
cm = confusion_matrix(y_true, y_pred_classes)
cm_display = ConfusionMatrixDisplay(cm, display_labels=test_generator_aug.class_indices.keys())
cm_display.plot(xticks_rotation='vertical')
plt.title('Matrice de confusion')
plt.show()